In [1]:
import pandas as pd
import numpy as np
import random

Резервуарна вибірка (Reservoir Sampling) — це стохастичний алгоритм, розроблений для ефективного відбору 
k елементів з множини невідомого або надзвичайно великого розміру n, де n може бути невизначеним або нескінченним.

In [7]:
def reservoir_sampling(stream, k):
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir


stream = range(1000000)
sample = reservoir_sampling(stream, 10)
sample

[529854, 971267, 821430, 787306, 3335, 638715, 516280, 377733, 489312, 178603]

In [2]:
def reservoir_sampling(path, k, chunksize=100000):
    """
    Виконує reservoir sampling (випадкову вибірку фіксованого розміру) з великого CSV-файлу, 
    не завантажуючи його повністю у пам'ять.

    Алгоритм гарантує, що кожен рядок у файлі має однакову ймовірність потрапити 
    у фінальну вибірку, навіть якщо розмір CSV заздалегідь невідомий.

    Параметри
    ----------
    path : str
        Шлях до CSV-файлу.
    k : int
        Кількість елементів у вибірці (розмір резервуару).
    chunksize : int, optional (default=100000)
        Кількість рядків, що зчитуються з файлу за один раз.

    Повертає
    -------
    pandas.DataFrame
        DataFrame, що містить `k` випадково відібраних рядків з усього CSV.

    Примітки
    --------
    - Цей метод корисний для обробки дуже великих наборів даних, 
      які не поміщаються у пам’ять цілком.
    - Використовує рівноймовірне заміщення на основі алгоритму Reservoir Sampling.
    - Для швидкодії використовується заміна рядків через `iloc`, без створення нових об’єктів.
    """
    reservoir = None
    total_rows = 0

    for chunk in pd.read_csv(path, chunksize=chunksize):
        chunk_len = len(chunk)

        if total_rows < k:
            # Поки не заповнили резервуар — просто додаємо
            if reservoir is None:
                reservoir = chunk.copy()
            else:
                reservoir = pd.concat([reservoir, chunk], ignore_index=True)
            total_rows += chunk_len

            # якщо перевищили k, скорочуємо
            if len(reservoir) > k:
                reservoir = reservoir.sample(n=k, replace=False, random_state=None)
        else:
            # Генеруємо випадкові індекси для можливих замін
            for i in range(chunk_len):
                j = random.randint(0, total_rows + i)
                if j < k:
                    reservoir.iloc[j] = chunk.iloc[i]
            total_rows += chunk_len

    return reservoir.reset_index(drop=True)

In [5]:
df = reservoir_sampling('data/final_proj_data.csv', k=1000)

In [6]:
df

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,y
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,qsDp1PS,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,1015.0,7.0,NaN,NaN,NaN,...,0VTT3Cb,LM8l689qOp,NaN,ELof,szEZ,RAYp,55YFVY9,am7c,NaN,0
2,NaN,NaN,NaN,NaN,NaN,1778.0,7.0,NaN,NaN,NaN,...,gkfqCcD,LM8l689qOp,NaN,kG3k,kwS7,RAYp,TCU50_Yjmm6GIBZ0lL_,am7c,NaN,0
3,NaN,NaN,NaN,NaN,NaN,630.0,7.0,NaN,NaN,NaN,...,Z2_IPGf,LM8l689qOp,NaN,ELof,WqMG,vJ_w8kB,WfJ2BB2SFSqauljlfOB,mj86,NaN,0
4,NaN,NaN,NaN,NaN,NaN,1288.0,7.0,NaN,NaN,NaN,...,8vXEsaq,LM8l689qOp,NaN,ELof,szEZ,RAYp,F2FyR07IdsN7I,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,749.0,7.0,NaN,NaN,NaN,...,BlJqCcD,LM8l689qOp,NaN,NaN,me1d,RAYp,F2FyR07IdsN7I,NaN,NaN,0
996,NaN,NaN,NaN,NaN,NaN,1155.0,7.0,NaN,NaN,NaN,...,5PZ9hga,LM8l689qOp,NaN,kG3k,xb3V,nIGXDli,F2FyR07IdsN7I,am7c,NaN,0
997,NaN,NaN,NaN,NaN,NaN,700.0,7.0,NaN,NaN,NaN,...,M7Q52Lp,LM8l689qOp,NaN,ELof,7aLG,RAYp,55YFVY9,mj86,NaN,0
998,NaN,NaN,NaN,NaN,NaN,1407.0,7.0,NaN,NaN,NaN,...,503RDbC,LM8l689qOp,NaN,ELof,Qcbd,RAYp,55YFVY9,am7c,NaN,0
